## Clean your deployed model endpoint content
-----

#### In this notebook, we will parse through the existing endpoint.json to delete all of the existing endpoints once you are done with running your respective benchmarking tests.

***If you are with running all of the tests, and want to delete the existing endpoints, run this notebook.***

In [2]:
## Import all necessary libraries
import json
import boto3
import logging
from fmbt.globals import *
from fmbt.utils import *

Loaded config: {'general': {'name': 'mistral-llama-7b-tgi-inf2-v1', 'model_name': 'mistral7b'}, 'aws': {'region': 'us-east-1', 'sagemaker_execution_role': 'arn:aws:iam::218208277580:role/service-role/AmazonSageMaker-ExecutionRole-20230911T184036', 'bucket': 'fmbt2039test'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompts', 'all_prompts_file': 'all_prompts.csv', 'metrics_dir': 'metrics_{datetime}', 'models_dir': 'models_{datetime}', 'metadata_dir': 'metadata'}, 's3_read_data': {'read_bucket': 'fmbt-read2039', 'scripts_prefix': 'scripts', 'source_data_prefix': 'source_data', 'tokenizer_prefix': 'tokenizer', 'prompt_template_dir': 'prompt_template', 'prompt_template_file': 'prompt_template.txt'}, 'run_steps': {'0_setup.ipynb': False, '1_generate_data.ipynb': False, '2_deploy_model.ipynb': False, '3_run_inference.ipynb': False, '4_model_metric_analysis.ipynb': False, '5_cleanup.ipynb': False}, 'datasets': [{'language': 'en', 'min_length_in_tokens': 1, 'max_length_in_tokens'

In [3]:
## Set your logger to display all of the endpoints being cleaned
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Load the config file to extract our endpoint.json file and its respective file path
config = load_config(CONFIG_FILE)

In [4]:
## Refer to the file path for the endpoint
endpoint_file_path = f"s3://{config['aws']['bucket']}/{ENDPOINT_LIST_PATH}"
endpoint_file_path

endpoint_info_list = json.loads(get_s3_object(endpoint_file_path))
logger.info(f"found information for {len(endpoint_info_list)} endpoints")
logger.info(json.dumps(endpoint_info_list, indent=2))

[2024-02-03 11:51:16,098] p5985 {3802923972.py:6} INFO - found information for 1 endpoints
[2024-02-03 11:51:16,099] p5985 {3802923972.py:7} INFO - [
  {
    "experiment_name": "mistral-7b-g5-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0",
    "endpoint": {
      "EndpointName": "lmistral7b-g5-xlarge-1706924687",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:218208277580:endpoint/lmistral7b-g5-xlarge-1706924687",
      "EndpointConfigName": "lmistral7b-g5-xlarge-1706924687",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04",
              "ResolvedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference@sha256:2739b630b95d8a95e6b4665e66d8243dd43b99c4fdb865feff13aab9c1da06eb",
              "ResolutionTime": "2024-02-02 20:44

In [5]:
## initialize a sagemaker client
sm_client = boto3.client("sagemaker")
# Iterate over the endpoint_info_list and mark the items for deletion
for item in endpoint_info_list:   
    
    ## Extract the endpoint name from the deployed model configuration
    ep_name = item['endpoint']["EndpointName"]
    try:
        ## Describe the model endpoint 
        logger.info(f"Going to describing the endpoint -> {ep_name}")
        resp = sm_client.describe_endpoint(EndpointName=ep_name)
        
        ## If the given model endpoint is in service, delete it 
        if resp['EndpointStatus'] == 'InService':
            logger.info(f"going to delete {ep_name}")
            ## deleting the model endpoint
            sm_client.delete_endpoint(EndpointName=ep_name)
            logger.info(f"deleted {ep_name}")
    except Exception as e:
        logger.error(f"error deleting endpoint={ep_name}, exception={e}")

[2024-02-03 11:51:18,600] p5985 {707195403.py:10} INFO - Going to describing the endpoint -> lmistral7b-g5-xlarge-1706924687


[2024-02-03 11:51:18,917] p5985 {707195403.py:20} ERROR - error deleting endpoint=lmistral7b-g5-xlarge-1706924687, exception=An error occurred (ValidationException) when calling the DescribeEndpoint operation: Could not find endpoint "lmistral7b-g5-xlarge-1706924687".
